# Optimizing Softmax Regression with MCMC

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
import emcee
import corner

## Softmax Regression

In [2]:
def softmax(z):                                        # define the softmax function
            z -= np.max(z)                             # for numerical stability
            return (np.exp(z).T / np.sum(np.exp(z),axis=1)).T
    
def one_hot(y, C):                                     # define the one-hot encoding for labels
            return (np.arange(C) == y[:, None]).astype(float)

class SoftmaxRegression:
    def __init__(self, add_bias=True):
        self.add_bias = add_bias
    
    def fit(self, x, y, optimizer):
        if self.add_bias:
            N = x.shape[0]
            x = np.column_stack([x,np.ones(N)])
        N,D = x.shape
        C = np.max(y) + 1
        w0 = np.zeros((D, C))               # initialize the weights to 0, note that the dimension is D*C
        # optimize using MCMC
        if isinstance(optimizer, BayesianInferenceMCMC):
            def log_likelihood(w_flat, x, y):
                w_og = w_flat.reshape(x.shape[1],np.max(y) + 1)
                yh = softmax(np.dot(x, w_og))                  # predictions, i.e. the probabilities for each label class
                log_like = np.sum(np.log(yh.max(axis=1)))   # log likelihood is the sum of the logs of all probabilities
                
                if math.isnan(log_like):
                    global dotxw 
                    dotxw = np.dot(x, w_og)
                    global nanyh 
                    nanyh = yh
                    #return -np.inf
                
                return log_like
            def log_prior(w_flat):
                return 0.0                                  # no specific preference for the parameters
            def log_post(w_flat, x, y):
                return log_prior(w_flat) + log_likelihood(w_flat, x, y)
            self.w = optimizer.run(log_post, x, y, w0)      # run the optimizer to get the optimal weights
        # optimize using Gradient Descent
        if isinstance(optimizer, ImprovedGradientDescent):
            def gradient(x, y, w):                          # define the gradient function
                yh = softmax(np.dot(x, w))                  # predictions
                y = one_hot(y, C)                           # labels with one-hot encoding
                N, D = x.shape
                grad = np.dot(x.T, (yh - y))/N              # divide by N because cost is mean over N points
                return grad
            self.w = optimizer.run(gradient, x, y, w0)      # run the optimizer to get the optimal weights
        return self
    
    def predict(self, x):
        if x.ndim == 1:
            x = x[:, None]
        if self.add_bias:
            N = x.shape[0]
            x = np.column_stack([x,np.ones(N)])
        yh = softmax(np.dot(x,self.w))                  #predict output
        return yh

In [3]:
# note that the output of the Softmax Regression is the probabilities for the label classes
# so we need to convert the output into the label classes with the highest probabilities for the classification purpose

def to_classlabel(z):                                   # convert the output class probabilities to be class labels
    return z.argmax(axis=1)

## Optimizer 1: Bayesian inference with MCMC

In [4]:
class BayesianInferenceMCMC:
    def __init__(self, visualization=False):
        self.visualization = visualization
            
    def run(self, log_post, x, y, w):
        num_iter = 2000
        w_flat = w.reshape(-1)
        ndim = len(w_flat) # number of parameters
        nwalkers = 50
        initial_pos = w_flat + 0.01 * np.random.randn(nwalkers, ndim)
        sampler = emcee.EnsembleSampler(nwalkers, ndim, log_post, args=(x, y))
        state = sampler.run_mcmc(initial_pos, num_iter, progress=True)
        sampler.reset()
        sampler.run_mcmc(state, num_iter, progress=True)
        w_flat = np.median(sampler.flatchain, axis=0)
        w = w_flat.reshape(x.shape[1],np.max(y) + 1)
        return w

## Optimizer 2: Gradient Descent (for comparison)

In [5]:
def create_mini_batches(x, y, batch_size): 
    mini_batches = []
    data = np.hstack((x, y[:,None]))
    np.random.shuffle(data) 
    n_minibatches = math.ceil(data.shape[0] // batch_size) 
    data = np.array_split(data, n_minibatches)
    mini_batches = []
    for batch_data in data:
        x_mini = batch_data[:, :-1]
        y_mini = batch_data[:, -1]
        mini_batches.append((x_mini, y_mini))
    return mini_batches

class ImprovedGradientDescent:
    def __init__(self, learning_rate=0.1, max_iters=1e4, epsilon=1e-8, record_history=False, momentum=0.5, batch_size=20):
        self.learning_rate = learning_rate
        self.max_iters = max_iters
        self.record_history = record_history
        self.epsilon = epsilon
        self.momentum = momentum
        self.batch_size = batch_size
        if record_history:
            self.w_history = []                         #to store the weight history for visualization
            
    def run(self, gradient_fn, x, y, w):
        grad = np.inf
        grad_old = 0
        t = 1
        accuracy = 0
        accuracy_best = 0
        num_worse = 0
        w_best = w
        while np.linalg.norm(grad) > self.epsilon and t < self.max_iters:
            mini_batches = create_mini_batches(x, y, self.batch_size) 
            for mini_batch in mini_batches: 
                x_mini, y_mini = mini_batch
                grad = self.momentum*grad_old + (1-self.momentum)*gradient_fn(x_mini, y_mini, w)  # compute the gradient with present weight and momentum
                w = w - self.learning_rate * grad       # weight update step
                grad_old = grad                         # the current gradient is the old gradient for the next iteration
            if self.record_history:
                self.w_history.append(w)
            accuracy = (to_classlabel(softmax(np.dot(x, w))) == y).astype(float).mean()
            if accuracy > accuracy_best:
                num_worse = 0
                accuracy_best = accuracy
                w_best = w
            else:
                num_worse += 1
            if num_worse >= 20:
                break
            t += 1
        w = w_best
        return w

## Performance Analysis 1: Iris dataset

In [6]:
from sklearn import datasets
import time

In [7]:
iris = datasets.load_iris()

iris_data = np.hstack((iris['data'],iris['target'][:,None]))
np.random.shuffle(iris_data)

train = iris_data[:(len(iris_data)*3//4)]
train_x = train[:,:-1]
train_y = train[:,-1].astype(int)

test = iris_data[(len(iris_data)*3//4):]
test_x = test[:,:-1]
test_y = test[:,-1].astype(int)

In [8]:
start = time.time()
regressor_iris_GD = SoftmaxRegression().fit(train_x,train_y, 
                                            ImprovedGradientDescent(learning_rate=0.01,momentum=0.9, batch_size=1))
stop = time.time()
train_time = stop - start
test_yh = regressor_iris_GD.predict(test_x)
test_accuracy = (to_classlabel(test_yh) == test_y).astype(float).mean()
print("The training time is", train_time)
print("The test accuracy is", test_accuracy)

The training time is 0.13334107398986816
The test accuracy is 1.0


In [9]:
start = time.time()
regressor_iris_GD = SoftmaxRegression().fit(train_x,train_y, 
                                            BayesianInferenceMCMC())
stop = time.time()
train_time = stop - start
test_yh = regressor_iris_GD.predict(test_x)
test_accuracy = (to_classlabel(test_yh) == test_y).astype(float).mean()
print("The training time is", train_time)
print("The test accuracy is", test_accuracy)

  0%|                                                                                         | 0/2000 [00:00<?, ?it/s]<ipython-input-2-ec91cddf4647>:3: RuntimeWarning: invalid value encountered in true_divide
  return (np.exp(z).T / np.sum(np.exp(z),axis=1)).T
  1%|▉                                                                              | 23/2000 [00:00<00:06, 303.44it/s]


ValueError: Probability function returned NaN

In [10]:
def softmax_1(z):                                        # define the softmax function
            z -= np.max(z)                             # for numerical stability
            return (np.exp(z).T / np.sum(np.exp(z),axis=1)).T
def softmax_2(z):                                        # define the softmax function
            z -= np.max(z)                             # for numerical stability
            return (np.exp(z).T).T
def softmax_3(z):                                        # define the softmax function
            z -= np.max(z)                             # for numerical stability
            return (np.sum(np.exp(z),axis=1)).T
def softmax_4(z):                                        # define the softmax function
            z -= np.max(z)                             # for numerical stability
            return (z).T
print(softmax_1(dotxw)[0])
print(softmax_2(dotxw)[0])
print(softmax_3(dotxw)[0])
print(softmax_4(dotxw)[0])

[0. 0. 1.]
[0.00000000e+000 0.00000000e+000 1.67660225e-309]
1.6766022543462e-309
[-1044.78584644  -425.75926478  -655.60185217 -1069.88175956
 -1094.5688852   -609.98090529 -1139.95182974  -754.06423108
  -723.24604454  -777.28641417  -530.79167969  -459.17629926
  -521.31348446  -538.13961189 -1082.54317771  -768.42539893
  -533.20909575  -707.37078343 -1089.12429284  -555.06682266
 -1092.32705665  -694.51524709  -496.51318612  -619.61981674
 -1116.09292577  -716.020757   -1083.86374974  -567.79226281
  -535.70265494 -1089.40036673  -564.81263004 -1078.60159946
 -1045.62514726  -555.87812084  -701.47452967 -1081.51982764
 -1067.61032119 -1058.87940215 -1096.63673341  -644.73060116
 -1004.55187948  -693.09317847  -671.96684623 -1095.90899812
 -1094.8254182   -628.08852298 -1071.91999796  -739.97067538
 -1103.8731001   -787.69754203  -713.58380004  -737.90282716
  -512.51854341  -721.83344787 -1068.90974516  -734.34042658
  -589.14172609  -668.11729262  -493.43044957 -1099.00019641
  -

<ipython-input-10-243808745497>:3: RuntimeWarning: invalid value encountered in true_divide
  return (np.exp(z).T / np.sum(np.exp(z),axis=1)).T
